# Chains using LangGraph

This notebook demonstrates how to creating a chain using LangGraph with `ToolNode` and `tools_condition`.

In [ ]:
from typing import Annotated, TypedDict
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
# Define a simple tool
@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

tools = [multiply]

# Initialize LLM with tools binding
llm = ChatGroq(model="llama3-8b-8192")
llm_with_tools = llm.bind_tools(tools)

In [ ]:
# Define State
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
# Define Chatbot Node
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [ ]:
# Build the Graph
builder = StateGraph(State)

# Add nodes
builder.add_node("chatbot", chatbot)
builder.add_node("tools", ToolNode(tools))

# Add edges
builder.add_edge(START, "chatbot")
builder.add_conditional_edges("chatbot", tools_condition)
builder.add_edge("tools", "chatbot")

graph = builder.compile()

In [ ]:
# Visualize the graph
from IPython.display import Image, display
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(e)

In [ ]:
# Run the graph
res = graph.invoke({"messages": [("user", "What is 2 multiplied by 3?")]})
for m in res["messages"]:
    m.pretty_print()